# Nødvendige biblioteker

In [1]:
import requests
import ftplib
import pandas as pd
import numpy as np
import tqdm
import time
from io import BytesIO
from bs4 import BeautifulSoup as bs

# Opsætning af ftp serveradresse og bibliotek

In [2]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"

# Initialisering af lister
## Jeg skal finde ud af, hvordan jeg dropper dem igen, når jeg er færdig

In [3]:
dirs = []
sub_dir = []
filer = []
filer_navne = []
filer_size = []
sti_navn = []
indhold = []
suppe = []

# Login til ftp serveren og henter en liste over biblioteker

In [4]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)

# Bygger en liste med underbibliotekter

In [5]:
for i in range (0, len(dirs), 1):
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 

# Bygger en liste med filnavne

In [6]:
for i in range (0, len(sub_dir), 1):
    ftp.cwd(sub_dir[i])
    ftp.dir(filer.append)

# Ekstrakt af filnavn

In [7]:
for i in range(0, len(filer)):
    filer_navne.append(filer[i][39:])

# Opbygning af hele stien og filnavn

In [9]:
for i in range(0, len(filer_navne)):
    sti_navn.append(ftp_dir+filer_navne[i][0:5]+'/'+filer_navne[i])

# Overførsel af de enkelte filer fra ftp://oda.ft.dk, og indholdet skrives til liste

In [10]:
for i in tqdm.tqdm(range(0, len(sti_navn))):
    r = BytesIO()
    ftp.retrbinary(f"RETR {sti_navn[i]}", r.write)
    indhold.append(r.getvalue())
    r.close()

100%|██████████████████████████████████████████████████████████████████████████████| 1489/1489 [04:04<00:00,  6.10it/s]


# BeautifulSoup - her kommer der korrekte danske karakterer, og der sker eet eller andet med XML-tags'ene

In [11]:
for i in tqdm.tqdm(range (0,len(indhold))):
    suppe.append(bs(indhold[i],"lxml"))

100%|██████████████████████████████████████████████████████████████████████████████| 1489/1489 [10:24<00:00,  2.38it/s]


# Dataframe skabes

In [13]:
df_suppe = pd.DataFrame(suppe, columns = ['metadata', 'indhold'])

C:\Users\kim\anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


# Der søges på de møder i salen, hvor ordet "sygeplejerske" har været nævnt
# Der returneres 512 møder i salen, hvilket svarer til ca. 35%

In [14]:
substring = 'sygeplejerske'
df_sygep = df_suppe[df_suppe.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]